<a href="https://colab.research.google.com/github/sigeek/fitting-covid-19/blob/main/models/LSTM/LSTM_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pickle
import datetime
from dateutil.relativedelta import relativedelta
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import Input
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
from numpy import array
import tensorflow as tf
import warnings
from tqdm import tqdm_notebook as tqdm
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly as py
from matplotlib import rcParams
import seaborn as sns
import matplotlib.pylab as plt
import math
import pickle
import os
import re
import pandas as pd
import folium
import math
import numpy as np
import matplotlib
from itertools import cycle
sns.set()
matplotlib.use('nbagg')
warnings.filterwarnings("ignore")

In [ ]:
!pip install --upgrade covsirphy

In [ ]:
import covsirphy as cs
# Download datasets
data_loader = cs.DataLoader("input")
jhu_data = data_loader.jhu()
population_data = data_loader.population()

In [ ]:
#WORLD: G20 countries
train = jhu_data.cleaned()
days_moving_average = 3
total_data = []
countries = ["Russia", "Italy", "United Kingdom", "France", "Spain", "Germany", "Poland", "Ukraine", "Czech Republic", "Netherlands"]

for country in countries:
  try:
    d, _ = jhu_data.records(country=country,auto_complement=False)
    s = cs.Scenario(jhu_data, population_data, country=country)
    diff = s.records_diff(variables=["Confirmed"], window=days_moving_average, show_figure=False)
    d["Country"] = country
    d["New Confirmed"] = diff.reset_index()["Confirmed"]
    d = d.loc[:, ["Date",	"Infected",	"Fatal",	"Recovered", "Country", "New Confirmed",	"Confirmed"]]
    total_data.append(d)
    print("inserted" + " " +str(country))
  except:
    print(country + ": no recovered")

train_df = pd.concat(total_data)
train_df.head()

In [ ]:
# ITALY
italy_data = pd.read_csv("https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-andamento-nazionale/dpc-covid19-ita-andamento-nazionale.csv")
days_moving_average = 3
train_df, _ = jhu_data.records("Italy",auto_complement=False)
s = cs.Scenario(jhu_data, population_data, country="Italy", auto_complement=False)
diff = s.records_diff(variables=["Confirmed"], window=days_moving_average, show_figure=False)
train_df["Hospitalized"] = italy_data['totale_ospedalizzati']
train_df["New Confirmed"] = diff.reset_index()["Confirmed"]
train_df = train_df.loc[:, ["Date",	"Infected",	"Fatal","Recovered", "New Confirmed",	"Confirmed", "Hospitalized"]]
train_df.head()

In [ ]:
def my_create_train_dataset(target, n_steps, train, pivot_date):
  train = train.query("ObservationDate<"+pivot_date)
  x, y =[], []
  sequence = list(train[target])
  for i in range(len(sequence)):
      end_ix = i + n_steps
      if end_ix > len(sequence) - 1:
          break
      seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
      if(seq_y != 0):
         x.append(seq_x)
         y.append(seq_y)
  return array(x), array(y)

def my_create_test_dataset(target, n_steps, train, pivot_date):
    train = train.query("ObservationDate<"+pivot_date)
    x = []
    sequence = train[target].values
    x.append(sequence[len(sequence)-n_steps:len(sequence)+1])
    return x

def pred(model, data):
    y_pred = model.predict(data)
    return y_pred

def my_forcast(model, data, start_date, num_days, n_steps):
    """
    Utility method for Forcasting
    model - trained model on Confirmed/Deaths data
    start_date - Starting date of forcasting
    num_days - Number of days for which forcasting is required
    """
    res_ = dict()
    for i in range(len(data)):
        res_[i] = []
    y_pred = pred(model, data)
    dates = []
    date1 = datetime.datetime.strptime(start_date, "%Y-%m-%d")

    for j in range(1, num_days+1):
        for i in range(len(data)):
            cur_window = list(data[i][0][1: n_steps+1])
            res_[i].append(cur_window[-1])
            cur_window.append(y_pred[i])
            data[i][0] = cur_window
        y_pred = pred(model, data)
        dates.append(date1.strftime("%Y-%m-%d"))
        date1 += relativedelta(days=1)
    res = pd.DataFrame(pd.DataFrame(res_).values.T)
    res.columns = dates
    return res

def my_convert_predictions(res_nc, test, pivot_date, end_date):
    test = test.query("Date>="+pivot_date)
    test = test.query("Date<="+end_date)
    pred_nc =[]
    for i in range(len(test)):
      date = datetime.datetime.strftime(test.iloc[i]['Date'].date(), format="%Y-%m-%d")
      pred_nc.append(res_nc[date])
    test["Hospitalized LSTM"] = pred_nc
    results = test
    return results

#WORLD
def pred(model, data):
    y_pred = model.predict(data)
    return y_pred


def forcast(model, data, start_date, num_days, n_steps, regs):
    """
    Utility method for Forcasting
    model - trained model on Confirmed/Deaths data
    start_date - Starting date of forcasting
    num_days - Number of days for which forcasting is required
    """
    res_ = dict()
    for i in range(len(data)):
        res_[i] = []
    y_pred = pred(model, data)
    dates = []
    date1 = datetime.datetime.strptime(start_date, "%Y-%m-%d")

    for j in range(1, num_days+1):
        for i in range(len(data)):
            cur_window = list(data[i][0][1: n_steps+1])
            res_[i].append(cur_window[-1])
            cur_window.append(y_pred[i])
            data[i][0] = cur_window
        y_pred = pred(model, data)
        dates.append(date1.strftime("%Y-%m-%d"))
        date1 += relativedelta(days=1)
    res = pd.DataFrame(pd.DataFrame(pd.DataFrame(res_).values.T))
    res.columns = dates
    res['Country/State'] = regs
    return res

def create_train_dataset(target, n_steps, train, pivot_date,
                         unique_regions, states_per_regions):
    train = train.query("ObservationDate<"+pivot_date)
    x, y =[], []
    for k in tqdm(range(len(unique_regions))):
        for state in states_per_regions[k]:
            temp = train[(train['Country/Region'] == unique_regions[k])]
            sequence = list(temp[target])
            for i in range(len(sequence)):
                end_ix = i + n_steps
                if end_ix > len(sequence) - 1:
                    break
                seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
                if(seq_y != 0):
                    x.append(seq_x)
                    y.append(seq_y)
    return array(x), array(y)

def create_test_dataset_new_c(target, n_steps, train,pivot_date, unique_regions,
                        states_per_regions):
    train = train.query("ObservationDate<"+pivot_date)
    x, regs = [], []
    for k in tqdm(range(len(unique_regions))):
        for state in states_per_regions[k]:
            temp = train[(train['Country/Region'] == unique_regions[k])]
            sequence = temp[target].values
            x.append(sequence[len(sequence)-n_steps:len(sequence)+1])
            regs.append((unique_regions[k],state))  
    return x, regs


def create_test_dataset(target, n_steps, train,pivot_date, unique_regions,
                        states_per_regions):
    """
    Function to create test dataset
    our supervised probem is now given last 7 days data predict 
    the no of cases for 8th day;
    target : 'Confirmed'/'Deaths'
    """

    train = train.query("ObservationDate<"+pivot_date)
    x, regs = [], []
    for k in tqdm(range(len(unique_regions))):
        for state in states_per_regions[k]:
            temp = train[(train['Country/Region'] == unique_regions[k])]
            sequence = temp[target].values
            x.append(sequence[len(sequence)-n_steps:len(sequence)+1])
            regs.append((unique_regions[k],state))  
    return x, regs

def list_per_region(train):
  # Creating list of all regions of all counntries
    unique_regions = train['Country/Region'].unique()
    states_per_regions = []
    for reg in tqdm(unique_regions):
        states_per_regions.append(
            train[train['Country/Region'] == reg]['Country/Region'].unique())
    return states_per_regions, unique_regions

def convert_predictions_nc(res_nc, test, pivot_date, end_date):
    test = test.query("Date>="+pivot_date)
    test = test.query("Date<="+end_date)
    index=dict()
    for i in range(len(res_nc)):
        index[res_nc.iloc[i]['Country/State'][0]]=i

    pred_c, pred_d, pred_nc=[], [], []
    for i in tqdm(range(len(test))):
        if(test.iloc[i]['Country/Region'] in index):
            loc=index[test.iloc[i]['Country/Region']]
            date = datetime.datetime.strftime(test.iloc[i]['Date'].date(), format="%Y-%m-%d")
            pred_nc.append(res_nc.iloc[loc][date]) 
    
    test['Infected'] = pred_nc
   
    res_regional=test
    res=test.drop(columns=['Country/Region','Date'])
    return res, res_regional

# Hyperparameter tuning

In [ ]:
#ITALY - only hospitalized
epochs = [20]
batch_sizes = [8,12,6]
n_hiddens = [50]
np.random.seed(42)
tf.random.set_seed(42)

epoch = 30
batch_size = 32
n_hidden = 36
num_days = 7

MAE, MSE, RMSE, R2 = {}, {}, {}, {}
target = "Hospitalized"

for epoch in epochs:
    for batch_size in batch_sizes:
        for n_hidden in n_hiddens:
          numdays = [7]
          for num_days in numdays:
            iteration = str(epoch) + "/" + str(batch_size) +  "/" + str(n_hidden) + "/" + str(num_days)
            print(iteration)

            train = train_df.rename(columns={'Fatal': 'Deaths', 'Date': 'ObservationDate'})
            covid_timeseries = train.copy()

            num_cols = ['Confirmed', 'Deaths']
            for col in num_cols:
                temp = [int(i) for i in train[col]]
                train[col] = temp

            # Create empty test set
            dates = list(train.ObservationDate)

            d = []
            for date in dates:
                d.append(
                    {
                        'Date': date
                    })

            test = pd.DataFrame(d)

            # Number of steps to look back
            n_steps = num_days

            # Number of days to forcast
            num_days = 10

            pivot_date = "'2021-03-15'"

            print('Hospitalized Cases')
            X_h, y_h = my_create_train_dataset('Hospitalized', n_steps, train, pivot_date)
            test_hospitalized = my_create_test_dataset('Hospitalized', n_steps, train, pivot_date)

            # Split the train data in to train and val data
            X_train_h, X_val_h, y_train_h, y_val_h = train_test_split(
                X_h, y_h, test_size=0.30, random_state=42)

            # Reshape data
            test_h = pd.DataFrame(test_hospitalized).values
            X_train_h = X_train_h.reshape((X_train_h.shape[0], 1, X_train_h.shape[1]))
            X_val_h = X_val_h.reshape((X_val_h.shape[0], 1,  X_val_h.shape[1]))

            X_test_h = test_h.reshape((test_h.shape[0], 1, test_h.shape[1]))

            # Initializing model components
            timesteps = X_train_h.shape[1]
            input_dim = X_train_h.shape[2]
            n_features = 1

            # Model for infected
            model_h = Sequential()
            model_h.add(LSTM(n_hidden, activation='relu', input_shape=(n_features,n_steps),return_sequences=True))
            model_h.add(LSTM(n_hidden, activation='relu'))
            model_h.add(Dense(1))
            model_h.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredLogarithmicError())
            callbacks = [ReduceLROnPlateau(monitor='val_loss', patience=5, verbose=1, factor=0.6),
                        EarlyStopping(monitor='val_loss', patience=20),
                        ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]
            hist=model_h.fit(X_train_h,y_train_h, epochs=epoch, batch_size=batch_size, validation_data=(X_val_h, y_val_h), verbose=2, shuffle=False,callbacks=callbacks)


            forcast_start_date = pivot_date.replace("'", "")
            res_new_confirmed = my_forcast(model_h, X_test_h, forcast_start_date, num_days, n_steps )
            begin_date = pivot_date
            end_date = f"'{res_new_confirmed.columns[-2:][0]}'"

            res_regional = my_convert_predictions(res_new_confirmed, test, begin_date, end_date)
            maes, mses, rmses, r2s = [], [], [], []
            Y_pred = res_regional
            Y_true = train_df
            temp = pd.merge(Y_true, Y_pred, on="Date")
            print(temp.columns)
            y, yhat = temp["Hospitalized LSTM"], temp[target]
            maes.append(metrics.mean_absolute_error(y, yhat))
            mses.append(metrics.mean_squared_error(y, yhat))
            rmses.append(np.sqrt(metrics.mean_squared_error(y, yhat)))
            r2s.append(metrics.r2_score(y,yhat))

            #PRINT
            MAE, MSE, RMSE, R2 = [], [], [], []
            Y_pred = res_regional

            predictions = []
            for i in range(Y_pred.shape[0]):
              prediction = str(list(Y_pred['Hospitalized LSTM'])[i])
              prediction = re.split('\s+', prediction)
              true_pred = float(prediction[1])
              predictions.append(true_pred)
            Y_pred["Hospitalized LSTM"] = predictions

            Y_true_before = Y_true[Y_true["Date"]<pivot_date]
            Y_true_after = Y_true[Y_true["Date"]>pivot_date]

            fig, axs = plt.subplots(1,1,figsize=(5, 4), constrained_layout=True);
            Y_pred.plot(x="Date", y="Hospitalized LSTM", label="H (LSTM forecast)", color="blue", ax=axs) 
            Y_true_before.plot(x="Date", y=target, label="H (reported)", ax=axs, color="magenta");
            Y_true_after.plot(x="Date", y=target, ax=axs, color="magenta",linestyle="dotted", label = "_ciao");
            axs.axvline(x=pivot_date, linestyle="--", color="grey")
            axs.legend(loc="upper left")
            axs.set_facecolor('white')

            axs.set_title(iteration + "\n Hospitalized")
  
            fig.show()

            temp = pd.merge(Y_true, Y_pred, on="Date")
            y, yhat = temp["Hospitalized LSTM"], temp[target]

            MAE.append(metrics.mean_absolute_error(y, yhat))
            MSE.append(metrics.mean_squared_error(y, yhat))
            RMSE.append(np.sqrt(metrics.mean_squared_error(y, yhat)))
            R2.append(metrics.r2_score(y,yhat))

            print("MAE", np.mean(MAE))
            print("MSE", np.mean(MSE))
            print("RMSE", np.mean(RMSE))
            print("R2", np.mean(R2))

In [ ]:
#WORLD
epochs = [30] #, 50, 70]
batch_sizes = [8] #[8, 12, 14]
n_hiddens = [50]
np.random.seed(42)
tf.random.set_seed(42)

epoch = 30
batch_size = 32
n_hidden = 36
num_days = 5

MAE, MSE, RMSE, R2 = {}, {}, {}, {}
target = "Infected"
countries = train_df["Country"].unique()

for epoch in epochs:
    for batch_size in batch_sizes:
        for n_hidden in n_hiddens:
            numdays = [5]
            for num_days in numdays:
              iteration = str(epoch) + "/" + str(batch_size) +  "/" + str(n_hidden) + "/" + str(num_days)
              print(iteration)

              train = train_df.rename(columns={'Country': 'Country/Region','Fatal': 'Deaths', 'Date': 'ObservationDate'})
              covid_timeseries = train.copy()

              num_cols = ['Confirmed', 'Deaths']
              for col in num_cols:
                  temp = [int(i) for i in train[col]]
                  train[col] = temp

              # Create empty test set
              countries = train["Country/Region"].unique()
              dates = list(train[train["Country/Region"] == countries[0]].ObservationDate)

              # Create empty test set
              d = []
              for country in countries:
                  for date in dates:
                      d.append(
                          {
                              'Country/Region': country,
                              'Date': date,
                          }
                      )
              test = pd.DataFrame(d)

              states_per_regions, unique_regions = list_per_region(test)

              # Number of steps to look back
              n_steps = num_days

              # Number of days to forcast
              num_days = 10

              pivot_date = "'2021-03-20'"


              print('Infected Cases')
              X_i, y_i = create_train_dataset('Infected', n_steps, train, pivot_date, unique_regions, states_per_regions)
              test_infected, regs = create_test_dataset('Infected', n_steps, train, pivot_date, unique_regions, states_per_regions)

              print('Recovered Cases')
              X_r, y_r = create_train_dataset('Recovered', n_steps, train, pivot_date, unique_regions, states_per_regions)
              test_recovered, regs = create_test_dataset('Recovered', n_steps, train, pivot_date, unique_regions, states_per_regions)

              print('Deaths Cases')
              X_d, y_d = create_train_dataset('Deaths', n_steps, train, pivot_date, unique_regions, states_per_regions)
              test_deaths, regs = create_test_dataset('Deaths', n_steps, train, pivot_date, unique_regions, states_per_regions)
              print('Finished preparing datasets.')

              # Split the train data in to train and val data
              X_train_d, X_val_d, y_train_d, y_val_d = train_test_split(
                  X_d, y_d, test_size=0.30, random_state=42)
              X_train_i, X_val_i, y_train_i, y_val_i = train_test_split(
                  X_i, y_i, test_size=0.30, random_state=42)
              X_train_r, X_val_r, y_train_r, y_val_r = train_test_split(
                  X_r, y_r, test_size=0.30, random_state=42)

              # Reshape data

              test_d = pd.DataFrame(test_deaths).values
              test_r = pd.DataFrame(test_recovered).values
              test_i = pd.DataFrame(test_infected).values


              # Reshapping the Confirmed data for LSTM

              X_train_i = X_train_i.reshape((X_train_i.shape[0], 1, X_train_i.shape[1]))
              X_val_i = X_val_i.reshape((X_val_i.shape[0], 1,  X_val_i.shape[1]))

              X_train_r = X_train_r.reshape((X_train_r.shape[0], 1, X_train_r.shape[1]))
              X_val_r = X_val_r.reshape((X_val_r.shape[0], 1,  X_val_r.shape[1]))

              X_train_d = X_train_d.reshape((X_train_d.shape[0], 1, X_train_d.shape[1]))
              X_val_d = X_val_d.reshape((X_val_d.shape[0], 1,  X_val_d.shape[1]))

              X_test_i = test_i.reshape((test_i.shape[0], 1, test_i.shape[1]))
              X_test_r = test_r.reshape((test_r.shape[0], 1, test_r.shape[1]))
              X_test_d = test_d.reshape((test_d.shape[0], 1, test_d.shape[1]))

              # Initializing model components
              timesteps = X_train_i.shape[1]
              input_dim = X_train_i.shape[2]
              n_features = 1

              # Model for Death cases
              #model_d = Sequential()
              #model_d.add(LSTM(50, activation='relu', input_shape=(n_features,n_steps),return_sequences=True))
              #model_d.add(LSTM(50, activation='relu'))
              #model_d.add(Dense(batch_size)) #MLP
              #model_d.add(Dense(1))
              #model_d.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredLogarithmicError())
              #callbacks = [ReduceLROnPlateau(monitor='val_loss', patience=5, verbose=1, factor=0.6),
                #                EarlyStopping(monitor='val_loss', patience=20),
                #              ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]
              #hist=model_d.fit(X_train_d,y_train_d, epochs=epoch, batch_size=batch_size, validation_data=(X_val_d, y_val_d), verbose=2, shuffle=False,callbacks=callbacks)

              # Model for new confirmed cases
              model_nc = Sequential()
              model_nc.add(LSTM(n_hidden, activation='relu', input_shape=(n_features,n_steps),return_sequences=True))
              model_nc.add(LSTM(n_hidden, activation='relu'))
              model_nc.add(Dense(batch_size)) #MLP
              model_nc.add(Dense(1))

              model_nc.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredLogarithmicError())
              callbacks = [ReduceLROnPlateau(monitor='val_loss', patience=5, verbose=1, factor=0.6),
                        EarlyStopping(monitor='val_loss', patience=20),
                         ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]
              hist=model_nc.fit(X_train_i,y_train_i, epochs=epoch, batch_size=batch_size, validation_data=(X_val_i, y_val_i), verbose=2, shuffle=False,callbacks=callbacks)


              forcast_start_date = pivot_date.replace("'", "")
              #     res_confirmed=forcast(model_c,X_test_c,forcast_start_date, num_days, n_steps, regs)
              #     res_deaths=forcast(model_d,X_test_d,forcast_start_date, num_days, n_steps, regs)
              res_new_confirmed = forcast(model_nc,X_test_i,forcast_start_date, num_days, n_steps, regs)

              begin_date = pivot_date
              end_date = f"'{res_new_confirmed.columns[-2:][0]}'"

              #     sub, res_regional = convert_predictions_df(res_confirmed, res_deaths, res_new_confirmed, test,begin_date, end_date)
              sub, res_regional = convert_predictions_nc(res_new_confirmed, test, begin_date, end_date)
              res_regional = res_regional.rename(columns={"Infected": "Infected LSTM"})
              maes, mses, rmses, r2s = [], [], [], []
              country = "Italy"
              #     try:
              Y_pred = res_regional[res_regional["Country/Region"] == country]
              Y_true = train_df[train_df["Country"] == country]
              temp = pd.merge(Y_true, Y_pred, on="Date")
              y, yhat = temp[target + " LSTM"], temp[target]
              maes.append(metrics.mean_absolute_error(y, yhat))
              mses.append(metrics.mean_squared_error(y, yhat))
              rmses.append(np.sqrt(metrics.mean_squared_error(y, yhat)))
              r2s.append(metrics.r2_score(y,yhat))

              MAE, MSE, RMSE, R2 = [], [], [], []
              target = "Infected"
              # res_regional = res_regional.rename(columns={"Confirmed": "Confirmed_LSTM", "New Confirmed": "New Confirmed_LSTM", "Deaths": "Deaths_LSTM"})
              # res_regional = res_regional.rename(columns={"New Confirmed": "New Confirmed_LSTM"})

              Y_pred = res_regional[res_regional["Country/Region"] == "Italy"]
              Y_true_before = Y_true[Y_true["Date"]<pivot_date]
              Y_true_after = Y_true[Y_true["Date"]>pivot_date]

              fig, axs = plt.subplots(1,1,figsize=(5, 4), constrained_layout=True);
              Y_pred.plot(x="Date", y="Infected LSTM", label="I (LSTM forecast)", color="blue", ax=axs) 
              Y_true_before.plot(x="Date", y=target, label="I (reported)", ax=axs, color="green");
              Y_true_after.plot(x="Date", y=target, ax=axs, color="green",linestyle="dotted", label = "_ciao");
              axs.axvline(x=pivot_date, linestyle="--", color="grey")
              axs.legend(loc="upper left")
              axs.set_facecolor('white')
              axs.set_title(iteration + "\n Infected")
  
              fig.show()

              temp = pd.merge(Y_true, Y_pred, on="Date")
              y, yhat = temp["Infected LSTM"], temp[target]
                  
              MAE.append(metrics.mean_absolute_error(y, yhat))
              MSE.append(metrics.mean_squared_error(y, yhat))
              RMSE.append(np.sqrt(metrics.mean_squared_error(y, yhat)))
              R2.append(metrics.r2_score(y,yhat))
                  
              print("MAE", np.mean(MAE))
              print("MSE", np.mean(MSE))
              print("RMSE", np.mean(RMSE))
              print("R2", np.mean(R2))

In [ ]:
# Infected forecast
# 30 ep, 8 batc + MLP, 5 window
pred_list = []
n_input_i = 5
n_features = 1

train_italy_i = train_df[train_df["Country"]=="Italy"]
dates = list(train_italy["Date"])
train_i = train_italy_i.drop(columns =["Fatal", "Recovered", "New Confirmed", "Confirmed", "Country"])
train_predicted_i =train_i.copy()

start_date = str(dates[-1])
date1 = datetime.datetime.strptime(start_date, "%Y-%m-%d %H:%M:%S")
last_index = train_predicted_i.index[-1]

pred_i = train_predicted_i.loc[last_index, "Infected"]
day = 0 
for day in range(690):
  day +=1
  ts_i = train_predicted_i["Infected"].values
  batch_i = ts_i[-n_input_i:].reshape((1 , n_features, n_input_i)) 
  date1 += relativedelta(days=1)
  date_pred = date1.strftime("%Y-%m-%d")
  date_pred = datetime.datetime.strptime(date_pred, "%Y-%m-%d")
  pred_i = model_nc.predict(batch_i)[0][0]
  new_index = last_index + day
  train_predicted_i.loc[new_index] = {"Infected": pred_i, "Date": date_pred}

Y_true_before_i = train_predicted_i[train_predicted_i["Date"]<"2021-03-30"]
Y_true_after_i = train_predicted_i[train_predicted_i["Date"]>"2021-03-30"]

In [ ]:
# Deaths forecast
pred_list = []
n_input_d = 10
n_features_d = 1

train_italy_d = train_df[train_df["Country"]=="Italy"]
dates_d = list(train_italy_d["Date"])
train_d = train_italy_d.drop(columns =["Infected", "Recovered", "New Confirmed", "Confirmed", "Country"])
train_predicted_d =train_d.copy()

start_date = str(dates[-1])
date1 = datetime.datetime.strptime(start_date, "%Y-%m-%d %H:%M:%S")
last_index = train_predicted_d.index[-1]

pred_d = train_predicted_d.loc[last_index, "Fatal"]
day =0 
for day in range(690):
  day +=1
  ts_d = train_predicted_d["Fatal"].values
  batch_d = ts_d[-n_input_d:].reshape((1 , n_features_d, n_input_d)) 
  date1 += relativedelta(days=1)
  date_pred = date1.strftime("%Y-%m-%d")
  date_pred = datetime.datetime.strptime(date_pred, "%Y-%m-%d")
  pred_d = model_d.predict(batch_d)[0][0]
  new_index =last_index + day
  train_predicted_d.loc[new_index] = {"Fatal": pred_d, "Date": date_pred}
  
Y_true_before_d = train_predicted_d[train_predicted_d["Date"]<"2021-03-30"]
Y_true_after_d = train_predicted_d[train_predicted_d["Date"]>"2021-03-30"]

In [ ]:
# Hospitalized forecast
n_input_h = 7
n_features = 1

train_italy_h = train_df.copy()
dates = list(train_italy_h["Date"])
train_h = train_italy_h.drop(columns =["Infected", "Fatal", "Recovered", "New Confirmed", "Confirmed"])
train_predicted_h =train_h.copy()

start_date = str(dates[-1])
date1 = datetime.datetime.strptime(start_date, "%Y-%m-%d %H:%M:%S")
last_index = train_predicted_h.index[-1]

pred_h = train_predicted_h.loc[last_index, "Hospitalized"]
day =0 
for day in range(600):
  day +=1
  ts_h = train_predicted_h["Hospitalized"].values
  batch_h = ts_h[-n_input_h:].reshape((1 , n_features, n_input_h)) 
  date1 += relativedelta(days=1)
  date_pred = date1.strftime("%Y-%m-%d")
  date_pred = datetime.datetime.strptime(date_pred, "%Y-%m-%d")
  pred_h = model_h.predict(batch_h)[0][0]
  new_index =last_index + day
  train_predicted_h.loc[new_index] = {"Hospitalized": pred_h, "Date": date_pred}

Y_true_before_h = train_predicted_h[train_predicted_h["Date"]<"2021-03-30"]
Y_true_after_h = train_predicted_h[train_predicted_h["Date"]>"2021-03-30"]

In [ ]:
#PLOT INFECTED
fig, axs = plt.subplots(1,1,figsize=(5,4), constrained_layout=True);
Y_true_before_i.plot(x="Date", y="Infected", label="I (reported)", ax=axs, color="green");
Y_true_after_i.plot(x="Date", y="Infected", ax=axs, color="green",linestyle="dotted", label = "I (forecast)");
axs.axvline(x="2021-03-30", linestyle="--", color="grey")
axs.set_title("LSTM model forecast")
axs.set_xlabel("Days")
axs.set_ylabel("Number of individuals")
axs.legend(loc="upper right")
axs.set_facecolor('white')
fig.show()

In [ ]:
#PLOT DEATHS FORECAST
# 20 ep, 16 batch, no MLP, time window 10
fig, axs = plt.subplots(1,1,figsize=(5,4), constrained_layout=True);
Y_true_before_d.plot(x="Date", y="Fatal", label="D (reported)", ax=axs, color="black");
Y_true_after_d.plot(x="Date", y="Fatal", ax=axs, color="black", linestyle="dotted",label = "D (forecast)");
axs.axvline(x="2021-03-30", linestyle="--", color="grey")
axs.set_title("LSTM model forecast")
axs.set_xlabel("Days")
axs.set_ylabel("Number of individuals")
axs.legend(loc="upper left")
axs.set_facecolor('white')
fig.show()